# Experiment 4: Isolate and Classify Wetland only Pixels
- New and Balanced dataset has needed to be created for this experiment.
- The dataset is created by using the following classes:
    - Wetland only training data
 
Want to mask out any pixels in the stack that are note classified as wetland.
The resulting image will be the one that the rf model is applied to.

what we needs to be done before doing the experiment:
- need the resulting classification from dtm classification from experiment 3
- need to find out how to create a mask from the classification

Workflow:
- going to follow the same workflow as the previous experiments
- the only difference is that the mask will be applied to the image stack before the rf model is used for prediction
- something to try: mask the stack from the very beginning and see if the model can predict the wetland pixels

```python
land_cover_colors = {
    "Bog": "#8B4513",          # Brown
    "Fen": "#808000",          # Olive Green
    "Marsh": "#008000",        # Medium Green
    "Salt marsh": "#9ACD32",   # Yellow-Green
    "Shallow water": "#ADD8E6",# Light Blue
    "Swamp": "#006400"         # Dark Green
}
```